In [2]:
!pip install langchain langchain_community pypdf

  Using cached pypdf-5.4.0-py3-none-any.whl.metadata (7.3 kB)
Using cached pypdf-5.4.0-py3-none-any.whl (302 kB)


In [5]:
from langchain_community.document_loaders import PyPDFLoader
path = './attention_is_all_you_need.pdf'
loader = PyPDFLoader(path)
docs = loader.load()
docs[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': './attention_is_all_you_need.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoog

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter()
chunks = splitter.split_documents(docs)
len(chunks)

16

In [7]:
!pip install langchain_huggingface langchain_faiss faiss-cpu

In [8]:
!pip install langchain-huggingface

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

c:\Users\MeesalaMrYeswanthKum\anaconda3\envs\ragenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunks,embeddings)

In [15]:
retriever = db.as_retriever()

In [3]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

In [5]:
prompt = """
you are a useful assistant. provide answers to the questions only from the provided context. If the user question is not in context, say that you don't know the answer.
If you find the answer, provide page number, title for the answer.
<context>
{context}
</context>
<question>
{input}
</question>

 """

qa_prompt = ChatPromptTemplate.from_template(prompt)

In [6]:
llm = ChatOllama(model='mistral')

In [21]:
stuff_chain = create_stuff_documents_chain(llm, qa_prompt)
retrieval_chain = create_retrieval_chain(retriever, stuff_chain)

In [22]:
answer = retrieval_chain.invoke({"input":"what is positional encoding?"})

In [24]:
answer['answer']

" Positional encoding is a method used to provide information about the position of a word or token within a sequence to a neural network model, as it doesn't inherently understand the order or positioning of words. In this paper, the authors use sinusoidal positional encoding, which encodes the absolute position of each word in the sequence by adding sine and cosine functions with different frequencies to the input vectors. This way, the model can learn to use the information about the relative positions of the words to better understand their meanings and context. The authors also mention that they tested using learned positional embeddings instead of sinusoids and saw nearly identical results."

In [11]:
prompt = '''
create a marketing pitch for a chocolate brand with vagon chocolate with 2% of protein and a base for which the shelf life is 3 months.
the name of the product is Vasechocs.
{input} 
'''

In [12]:
temp = ChatPromptTemplate.from_template(prompt)
chain = temp | llm

In [13]:
answer = chain.invoke({'input':'write a marketing pitch for Vasechocs'})

In [16]:
answer

AIMessage(content=" Title: Introducing Vasechocs - The Perfect Blend of Taste, Nutrition, and Quality!\n\nDear Valued Customers,\n\nWe are thrilled to present to you our latest culinary masterpiece, Vasechocs! A symphony of flavors that harmoniously combines the richness of chocolate with a surprising 2% protein content. This isn't just another chocolate bar; it's a delicious snack designed for those who appreciate taste and health in equal measure.\n\nVasechocs is more than just a name; it's a testament to our commitment to quality, innovation, and customer satisfaction. Our chocolates are made with the finest ingredients sourced from around the globe, ensuring a rich, creamy texture and an unforgettable taste experience.\n\nBut we didn't stop there. We understand that health is just as important as taste. That's why each Vasechocs bar contains 2% protein, providing you with a quick, convenient, and tasty source of essential nutrients.\n\nAnd here's the cherry on top: Each Vasechocs b

In [18]:
from IPython.display import display, Markdown
display(Markdown(answer.content))

 Title: Introducing Vasechocs - The Perfect Blend of Taste, Nutrition, and Quality!

Dear Valued Customers,

We are thrilled to present to you our latest culinary masterpiece, Vasechocs! A symphony of flavors that harmoniously combines the richness of chocolate with a surprising 2% protein content. This isn't just another chocolate bar; it's a delicious snack designed for those who appreciate taste and health in equal measure.

Vasechocs is more than just a name; it's a testament to our commitment to quality, innovation, and customer satisfaction. Our chocolates are made with the finest ingredients sourced from around the globe, ensuring a rich, creamy texture and an unforgettable taste experience.

But we didn't stop there. We understand that health is just as important as taste. That's why each Vasechocs bar contains 2% protein, providing you with a quick, convenient, and tasty source of essential nutrients.

And here's the cherry on top: Each Vasechocs bar has a shelf life of 3 months! This means you can stock up and enjoy our delicious chocolates anytime, anywhere, without worrying about expiry dates.

Vasechocs is more than a chocolate brand; it's a lifestyle choice for those who appreciate the finer things in life. So why wait? Treat yourself to the exquisite taste of Vasechocs today and indulge in a guilt-free chocolate experience like no other!

Stay tuned for more exciting flavors and offers coming soon. Remember, with Vasechocs, every moment is a moment to savor!

Best Regards,
[Your Company Name]